In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import PIL as Image
import os
from random import shuffle
import glob
import tensorflow as tf
from sklearn.utils import shuffle

C:\Users\Naman\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
import cv2
import os
from os import listdir
from os.path import isfile, join

In [ ]:
path="C:/Users/asd/Desktop/Winter Training/Classification_Assignment_Data"
cats=[]
dogs=[]
for i in range(1440):
    cats.append(path+"/cats/cat."+str(i)+'.jpg')
    dogs.append(path+"/dogs/dog."+str(i)+'.jpg')
print(np.shape(cats))
print(np.shape(dogs))
totalData=cats+dogs
print(np.shape(totalData))

In [ ]:
batchhsize=64


In [ ]:
labelDatadash=np.arange(2880)
labelDatadash[0:1440]=0
labelDatadash[1440:2880]=1
oneHotDash=np.eye(2)[labelDatadash]
labelData=np.reshape(oneHotDash,(2880,2))
print(labelData.shape)

In [ ]:
def unison_shuffle(a, b):
    X,Y=shuffle(a, b, random_state=0)
    return X, Y 
#     assert len(a) == len(b)
#     p = np.random.permutation(len(a))
#     return a[p], b[p]

In [ ]:
totalData,labelData=unison_shuffle(totalData,labelData)
print(np.shape(totalData))

In [ ]:
trainData=totalData[0:2304]
trainLabel=labelData[0:2304]
valData=totalData[2304:2880]
valLabel=labelData[2304:2880]

In [ ]:
print(trainData[6],trainLabel[1])


In [ ]:
def createBatch(start,batchSize,newSize):
    batchAddr=trainData[start:start+batchSize]
    batchData=[]
    for j in range(batchSize):
        img=cv2.imread(batchAddr[j])
        img.resize(newSize,newSize,3)
        batchData.append(img)
    batchLabel=trainLabel[start:start+batchSize]
    return batchData,batchLabel

In [ ]:
x = tf.placeholder(tf.float32, shape=[None,128,128,3])
y_ = tf.placeholder(tf.float32, shape=[None,2])

In [ ]:

defdef  weight_variableweight_v (shape):
    initial = tf.truncated_normal(shape, stddev=0.1)#truncated, anything outside -0.2 and 0.2 has a probability of occurence 0
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)#constant array of 0.1 with whatever shape we give to it
    return tf.Variable(initial)

In [ ]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [ ]:
W_conv1 = weight_variable([5, 5, 3, 32])#5X5 filter 3 are the in channel (gray-scale) and there are 32 feature maps
b_conv1 = bias_variable([32])#32 feature maps

In [ ]:
#x_image = tf.reshape(x, [-1, 28, 28, 1])
h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)#batchSizeX128X128X32. Here 64 is the batch size (assumed)
h_pool1 = max_pool_2x2(h_conv1)#max pooling batchSizeX64X64X32

In [ ]:
W_conv2 = weight_variable([5, 5, 32, 64])#batchSize64
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)#batchSizeX32X32X64

In [ ]:
W_fc1 = weight_variable([32 * 32 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 32*32*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [ ]:
keep_prob = tf.placeholder(tf.float32)#probability of retaining the neurons. 0=all neurons dropped, 1 means no neuron is dropped
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [ ]:
W_fc2 = weight_variable([1024, 2])
b_fc2 = bias_variable([2])

y = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [ ]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

In [ ]:
train_step = tf.train.RMSPropOptimizer(1e-2).minimize(cross_entropy)


In [ ]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
with tf.Session() as sess:#declaring tf.Session as sess
    sess.run(tf.global_variables_initializer())#initializing all the variables we used
    for j in range(10):
        for i in range(24):#number of iterations we are using
            batchData,batchLabel=createBatch(i*96,96,128)
            #print(batchData.shape)
            train_accuracy = accuracy.eval(feed_dict={x: batchData, y_: batchLabel, keep_prob: 1.0})#training accuracy calculated with no dropout probabilty
            print('step %d, training accuracy %g' % (i, train_accuracy))
            train_step.run(feed_dict={x: batchData, y_: batchLabel, keep_prob: 0.7})#dropout probabilty during training is kept 0.5
    print('test accuracy %g' % accuracy.eval(feed_dict={x:valData, y_: valLabel, keep_prob: 1.0}))#test accuracy